# Homework 3

* Assigned: 10/18 Thursday

* Due: 11/15 Thursday, 10:00 AM via. Instabase + Gradescope

* Value: 3.75% of your grade

In this assignment it's time to get real! You'll first flex your SQL muscles and perform analyses similar to HW2's by writing SQL and reflecting on the experience. You will then perform some normalization.

## Announcement

Homework 3 has two parts. __Part 1__ requires you to __complete and submit via Instabase__ by the token provided. We have setup the environment for you. Part 2 is a written task requiring a hard copy to be submitted on Gradescope.

__Please do not revise any existing block or variable without instruction.__ It may cause failure for running grading script, which will result in some grade deduction. Query quality and correctness will also be part of the grading rubric. We will grade mannually to allow some partial credits.

## Token Setup

In [54]:
# Your columbia uni that is used in SSOL
#
# IMPORTANT:  make sure this is consistent with the uni/alias used as your @columbia.edu email in SSOL
#
UNI = "wt2271"

# your instabase username (if you go to the instabase homepage, your username should be in the URL)
USER = "wesley"

# your repository name containing 
REPO = "my-repo"


## Database Connection

Run following command to connect to database. You should see "Connected to ..." for successful connection.

In [1]:
# connect to db in public repo
ib.connect_db('ib://ewu/w4111-public/databases/w4111')

Connected to: ib://ewu/w4111-public/databases/w4111


In [5]:
%%sql
select count(distinct(vendor_name)) from iowa where itemno = 3326

,count
0,1


We are using a 1mil-tuple sample** of the iowa liquor sales data for this assignment, with column names and datatype (in format of [name] [datatype]) as:

  - address char(256),
  - bottle_volume_ml integer,
  - category char(256),
  - category_name char(256),
  - city char(256),
  - county char(256),
  - county_number char(256),
  - date date,
  - im_desc char(256),
  - invoice_line_no char(256),
  - itemno integer,
  - name char(256),
  - pack integer,
  - sale_bottles integer,
  - sale_dollars double precision,
  - sale_gallons double precision,
  - sale_liters double precision,
  - state_bottle_cost double precision,
  - state_bottle_retail double precision,
  - store integer,
  - store_location char(256),
  - store_location_address char(256),
  - store_location_city char(256),
  - store_location_zip char(256),
  - vendor_name char(256),
  - vendor_no integer,
  - zipcode text
  
A description of column names can be found here: https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy.

You may run this query to have some ideas on the schema you are dealing with:

In [3]:
%%sql
select * from iowa limit 1;

,address,bottle_volume_ml,category,category_name,city,county,county_number,date,im_desc,invoice_line_no,...,state_bottle_cost,state_bottle_retail,store,store_location,store_location_address,store_location_city,store_location_zip,vendor_name,vendor_no,zipcode
0,1404 1ST AVE NE ...,375,1082900.0 ...,MISC. IMPORTED CORDIALS & LIQUEURS ...,CEDAR RAPIDS ...,Linn ...,57.0 ...,2013-08-05,Jagermeister Liqueur ...,S13729500031 ...,...,6.50999999999999979,9.76999999999999957,3894,POINT (-91.653699 41.987286) ...,1404 1ST AVE NE ...,CEDAR RAPIDS ...,52402 ...,Sidney Frank Importing Co. ...,192,52402


__Disclaimer: this course does not condone drinking, we are using this dataset because it is a common format for a sales transaction log in a silghtly more accessible domain than typical bank transactions__.

## Part I: SQL, the sequel

### Jupyter Notes: _Read these carefully_

* You **may** create new IPython notebook cells to use for e.g. testing, debugging, exploring, etc.- this is encouraged in fact!- **just make sure that you run the final cell to submit your results**
  * you can press shift+enter to execute to code in the cell that your cursor is in.
* When you see `In [*]:` to the left of the cell you are executing, this means that the code / query is _running_. Please wait for the execution to complete
    * **If the cell is hanging- i.e. running for too long: you can restart the kernel**
    * To restart kernel using the menu bar: "Kernel >> Restart >> Clear all outputs & restart"), then re-execute cells from the top
* _Have fun!_

__Now, please write the SQL query for each of the following questions.__

__Fill answers according to instruction in each question. Make sure to submit your query also (instruction given).__

Note: Some queries will take a few minutes to run on your virtual machine. If your query is running for more than ~10 minutes, you either did something wrong, 
or you may need to create tables with your temp tables to run the query more efficiently. 

For example: If you use the results of a sub-query multiple times, it 
can help to use SELECT ... INTO ... then reference that table.


### __(2 points) Q1.1: Which store had the most sales in terms of total records in the table?__

In [1]:
{"q1": {"count": 31784478, "query": "select store, sum(store) as record from iowa group by store order by record limit 1;", "store": 4829}, "q3": {"1": {"qty": 64368, "zipcode": 52402, "category_name": "VODKA 80 PROOF ..."}, "2": {"qty": 55597, "zipcode": 52240, "category_name": "VODKA 80 PROOF ..."}, "3": {"qty": 49893, "zipcode": 50314, "category_name": "VODKA 80 PROOF ..."}, "4": {"qty": 44763, "zipcode": 50320, "category_name": "VODKA 80 PROOF ..."}, "5": {"qty": 40812, "zipcode": 51501, "category_name": "VODKA 80 PROOF ..."}, "query": "with category_sale as (select zipcode, category_name, sum(sale_bottles) as total_bottles from iowa group by zipcode, category_name) select a.zipcode, b.category_name, a.max from (select max(total_bottles) as max, zipcode from category_sale group by zipcode) as a left join category_sale b on a.max = b.total_bottles and a.zipcode = b.zipcode order by max desc limit 5;"}, "q2": {"count": 1181, "vendor_no": 260, "query": "select vendor_no, count(vendor_no) from iowa where store = 4829  group by vendor_no order by count(vendor_no) desc limit 1;"}, "q5": {"query": "with store_avg as(select avg(state_bottle_cost/bottle_volume_ml) as cost_ml, store, itemno from iowa group by store, itemno) select itemno from store_avg group by itemno order by avg(cost_ml) desc limit 1;", "itemno": 927368}, "q4": {"count": 116, "query": "with tequila as (select count(distinct itemno) as count, store from iowa where im_desc like '%tequila%' group by store) select count(distinct i.zipcode) from tequila t left join iowa i on t.store = i.store where t.count > 1;"}, "q6": {"sql_cons": "", "python_cons": "", "python_pros": "as a general purpose programming language, python can achieve more functionality than sql", "sql_pros": "easier to write and understand when write queries talk to database."}, "user": "cherylxiao", "uni": "yx2444"}

{'q1': {'count': 31784478,
  'query': 'select store, sum(store) as record from iowa group by store order by record limit 1;',
  'store': 4829},
 'q3': {'1': {'qty': 64368,
   'zipcode': 52402,
   'category_name': 'VODKA 80 PROOF ...'},
  '2': {'qty': 55597, 'zipcode': 52240, 'category_name': 'VODKA 80 PROOF ...'},
  '3': {'qty': 49893, 'zipcode': 50314, 'category_name': 'VODKA 80 PROOF ...'},
  '4': {'qty': 44763, 'zipcode': 50320, 'category_name': 'VODKA 80 PROOF ...'},
  '5': {'qty': 40812, 'zipcode': 51501, 'category_name': 'VODKA 80 PROOF ...'},
  'query': 'with category_sale as (select zipcode, category_name, sum(sale_bottles) as total_bottles from iowa group by zipcode, category_name) select a.zipcode, b.category_name, a.max from (select max(total_bottles) as max, zipcode from category_sale group by zipcode) as a left join category_sale b on a.max = b.total_bottles and a.zipcode = b.zipcode order by max desc limit 5;'},
 'q2': {'count': 1181,
  'vendor_no': 260,
  'query': 'selec

In [9]:
%%sql
-- Complete your query here
select count(*), store from iowa group by store order by count desc limit 1

,count,store
0,8934,2633


In [56]:
# Please replace None with the result store and record number here
# submit query in string format
q1 = {
    'store': 2633,
    'count': 8934,
    'query': 'select count(*), store from iowa group by store order by count desc limit 1'
}

### __(2 points) Q1.2: At the store with the most total records (answer to Q1.1), what was the vendor number with most sale records? (The vendor that has the most records in the table?)__

In [ ]:
select vendor_no, count(vendor_no) from iowa where store = 4829  group by vendor_no order by count(vendor_no) desc limit 1

In [16]:
%%sql
-- Complete your query here
select vendor_no, count(*) from iowa where store = (
    select store from iowa group by store order by count(*) desc limit 1) 
    group by vendor_no order by count desc limit 1 

,vendor_no,count
0,260,1509
1,65,1427


In [57]:
# please replace None with the result vendor_no and according sum here
# submit query in string format
q2 = {
    'vendor_no': 260,
    'count': 1509,
    'query': 'select vendor_no, count(*) from iowa where store = (select store from iowa group by store order by count(*) desc limit 1) group by vendor_no order by count desc limit 1'
}

### __(3 points) Q1.3: For each zipcode, compute the single most purchased category_name by total sale_bottles. __

__Return the top 5 (zipcode, category_name) when sorted in descending order by the most purchased total sale_bottles.__

In [ ]:
with category_sale as 
(select zipcode, category_name, sum(sale_bottles) as total_bottles from iowa group by zipcode, category_name) 
select a.zipcode, b.category_name, a.max 
from (select max(total_bottles) as max, zipcode from category_sale group by zipcode) as a 
left join category_sale b on a.max = b.total_bottles and a.zipcode = b.zipcode 
order by max desc limit 5;

In [3]:
%%sql
-- Complete your query here
with temp as (select zipcode,category_name ,sum(sale_bottles) as qty from iowa group by zipcode, category_name)

select t.zipcode, t.qty, category_name
from (select zipcode, max(qty) as qty_max from temp group by zipcode) as t join temp on t.zipcode = temp.zipcode and t.qty=temp.qty
order by qty_max desc limit 5


UsageError: Cell magic `%%sql` not found.


In [58]:
# please replace None with the result accordingly here
# submit query in string format
q3 = {
    1: {'zipcode': 52402, 'category_name': "VODKA 80 PROOF", 'qty': 64368},
    2: {'zipcode': 52240, 'category_name': "VODKA 80 PROOF", 'qty': 55597},
    3: {'zipcode': 50314, 'category_name': "VODKA 80 PROOF", 'qty': 49893},
    4: {'zipcode': 50320, 'category_name': "VODKA 80 PROOF", 'qty': 44763},
    5: {'zipcode': 51501, 'category_name': "VODKA 80 PROOF", 'qty': 40812},
    'query': '''with temp as (select zipcode,category_name ,sum(sale_bottles) as qty from iowa group by zipcode, category_name)
select t.zipcode, t.qty, category_name
from (select zipcode, max(qty) as qty from temp group by zipcode) as t join temp on t.zipcode = temp.zipcode and t.qty=temp.qty
order by qty desc limit 5'''
}

### __(3 points) Q1.4: This problem has two steps, you only need to return the value from second step.__

__Compute the set of all liquors with the characters "tequila" (in lowercase) in its description (`im_desc`). The attribute `itemno` is the one that can serve as a unique identifier for a specific liquor.__

__Return the count of all distinct value of zipcode for all liquor stores that sold at least two of every tequila as defined in the previous sentence.__

(Optional: It is possible to rewrite the query in a form that can execute quite fast, but it will require you to create some temporary tables and/or experiment with alternate forms to express the problem. If you find the solution, feel free to submit it!)

In [43]:
%%sql
-- Complete your query here
with temp as (select distinct(zipcode)
      from iowa 
      where itemno in (select distinct(itemno) from iowa where im_desc like '%tequila%') 
      group by store, zipcode 
      having sum(sale_bottles) > 2)
select count(*)
from temp

,count
0,232


In [59]:
# please replace None with the according count here
# submit query in string format
q4 = {
    'count': 232,
    'query': '''with temp as (select distinct(zipcode)
      from iowa 
      where itemno in (select distinct(itemno) from iowa where im_desc like '%tequila%') 
      group by store, zipcode 
      having sum(sale_bottles) > 2)
select count(*)
from temp'''
}

In [2]:
"with tequila as (select count(distinct itemno) as count, store from iowa where im_desc like '%tequila%' group by store) 
select count(distinct i.zipcode) from tequila t left join iowa i on t.store = i.store where t.count > 1;"

"with tequila as (select count(distinct itemno) as count, store from iowa where im_desc like '%tequila%' group by store) select count(distinct i.zipcode) from tequila t left join iowa i on t.store = i.store where t.count > 1;"

### __(3 points) Q1.5:  __What is the item number (`itemno`) of the liquor with the highest state bottle cost, (as defined below), across all of iowa?

Let a liquor's _bottle cost_ for a given store $S_i$ be the state bottle retail cost per milliliter (mL) based on `state_bottle_retail` (this is the amount the store paid for each bottle of liquor ordered), averaged across all sales at store $S_i$. Note: The `bottle_volume_ml` field provides the volume of each bottle ordered in mL. Ignore the quantity of bottles sold in a sale.

Let a liquor's overall _state bottle cost_ be the average of all the per store costs, averaged across all stores in iowa.

For example: If a single store sells an item (say item no. 100), and has two coresponding records: 10 750 mL bottles of \$10 state bottle cost each (total cost: \$100), and later 1 bottle of a 1000 mL bottle of \$20 (total cost: \$20), the average is (10/750 + 20/1000)/2 = 0.01666...).

In [51]:
%%sql
-- Complete your query here
with temp as (select avg(state_bottle_retail/bottle_volume_ml) as unit_cost, store, itemno
from iowa 
group by store, itemno)

select avg(unit_cost), itemno from temp
group by itemno
order by avg(unit_cost) desc limit 1

,avg,itemno
0,11.5999999999999996,927368


In [60]:
# please replace None with the item number here
# submit query in string format
q5 = {
    'itemno': 927368,
    'query': '''with temp as (select avg(state_bottle_retail/bottle_volume_ml) as unit_cost, store, itemno
from iowa 
group by store, itemno)

select avg(unit_cost), itemno from temp
group by itemno
order by avg(unit_cost) desc limit 1'''
}

### __(3 points) Q1.6: Write a short paragraph about the main differences between writing Python code and writing SQL. List one pro and one con for each approach. __

In [61]:
q6 = {
    # put Python's pros and cons here
    # please write in the quotes
    'python_pros': "easy to do data wrangling and modify the atomic data at very low level",
    'python_cons': "not very effecient when we only care the result",
    'sql_pros': "the syntax is more human readable and natural",
    'sql_cons': "sometimes query logic is easy but we need complex query to implement just like double negation"
}

## Part I Submission

### Create your submission file¶

Run the following cell to create a results file for your homework

DO NOT MODIFY THE FOLLOWING CELL!!

In [62]:
import datetime
import json

script_path = '{0}/{1}/fs/Instabase%20Drive'.format(USER, REPO)

with ib.open('results'.format(script_path), "w") as f:
    result = dict(
        q1=q1,
        q2=q2,
        q3=q3,
        q4=q4,
        q5=q5,
        q6=q6,
        uni=UNI,
        user=USER
    )
    f.write(json.dumps(result))
    print "Result file created at: {0}".format(datetime.datetime.now())
  
    print
    print "Check your results: http://instabase.com/{0}/HW3/results".format(script_path)

Result file created at: 2018-11-12 21:18:15.792847

Check your results: http://instabase.com/wesley/my-repo/fs/Instabase%20Drive/HW3/results


Finally, Submit your __HW3 folder including the results file__ at the following URL:
    
https://www.instabase.com/apps/file-submission/cmd/submit/23eaec3c-154a-4898-848f-bf73b220582d


## Part II, Normalization

See Github page for more details. Submit __hard copy__ for Part 2 as instructed __before due__.

_**Remarks: Dataset sourced from https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy. 1mil tuples extracted and preprocessed to remove double quotation marks and apostrophes._